In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DateType, \
    StringType, TimestampType, DecimalType, IntegerType
from parsers import parse_line
import glob

spark = SparkSession.builder.getOrCreate()
spark.sparkContext.addPyFile("parsers.py")

In [2]:
from configreader import ConfigReader
from azure.storage.blob import BlobServiceClient

reader = ConfigReader("config.cfg", "azure-storage")
config = reader.get_config()

# Get Azure storage info from config
storage_acct_name = config["account_name"]
storage_acct_access_key = config["access_key"]
storage_container = config["container_name"]

# Set Spark Azure storage account and key
storage_acct_key_str = f"fs.azure.account.key.{storage_acct_name}.blob.core.windows.net"
spark.conf.set(storage_acct_key_str, storage_acct_access_key)

# Set base Spark filepath for container
container_base_path = f"​wasbs://{storage_container}@{storage_acct_name}.blob.core.windows.net"
mount_base_path 

# Set up container client
blob_service_client = BlobServiceClient(account_url=f"https://{storage_acct_name}.blob.core.windows.net", \
    credential=storage_acct_access_key)
container_client = blob_service_client.get_container_client(storage_container)
ingest_data_dir = "data"

# Set filetype
file_type = "txt"
file_suffix = f".{file_type}"
suffix_len = len(file_suffix)

# Get list of file names
blob_list = container_client.list_blobs(name_starts_with=ingest_data_dir)
cont_filepaths = [ blob.name for blob in blob_list if blob.name[-suffix_len:] == file_suffix ]

spark_filepath_list = [ f"{container_base_path}/{file}" for file in cont_filepaths ]
spark_filepath_str = ",".join(spark_filepath_list)
print(spark_filepath_str)

​wasbs://stock-proj-data@cdgreggstockproj.blob.core.windows.net/data/csv/2020-08-05/NYSE/part-00000-5e4ced0a-66e2-442a-b020-347d0df4df8f-c000.txt,​wasbs://stock-proj-data@cdgreggstockproj.blob.core.windows.net/data/csv/2020-08-06/NYSE/part-00000-214fff0a-f408-466c-bb15-095cd8b648dc-c000.txt,​wasbs://stock-proj-data@cdgreggstockproj.blob.core.windows.net/data/json/2020-08-05/NASDAQ/part-00000-c6c48831-3d45-4887-ba5f-82060885fc6c-c000.txt,​wasbs://stock-proj-data@cdgreggstockproj.blob.core.windows.net/data/json/2020-08-06/NASDAQ/part-00000-092ec1db-39ab-4079-9580-f7c7b516a283-c000.txt


In [8]:
raw = spark.sparkContext.textFile(f'{spark_filepath_str}')
parsed = raw.map(lambda line: parse_line(line)) 

IllegalArgumentException: java.net.URISyntaxException: Illegal character in scheme name at index 0: ​wasbs://stock-proj-data@cdgreggstockproj.blob.core.windows.net/data/csv/2020-08-05/NYSE/part-00000-5e4ced0a-66e2-442a-b020-347d0df4df8f-c000.txt

In [5]:
# Establish common event schema
schema = StructType([ \
    StructField('trade_dt', DateType(), True), \
    StructField('rec_type', StringType(), True), \
    StructField('symbol', StringType(), True), \
    StructField('exchange', StringType(), True), \
    StructField('event_tm', TimestampType(), True), \
    StructField('event_seq_nb', IntegerType(), True), \
    StructField('arrival_tm', TimestampType(), True), \
    StructField('trade_pr', DecimalType(17,14), True), \
    StructField('bid_pr', DecimalType(17,14), True), \
    StructField('bid_size', IntegerType(), True), \
    StructField('ask_pr', DecimalType(17,14), True), \
    StructField('ask_size', IntegerType(), True), \
    StructField('partition', StringType(), True) \
])      

# Create dataframe with parsed data and schema
df = spark.createDataFrame(parsed, schema)

In [6]:
df.show(10)

Py4JJavaError: An error occurred while calling o46.showString.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: /"spark_filepath_str"
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:213)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:307)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:303)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:57)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:307)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:303)
	at org.apache.spark.api.python.PythonRDD.getPartitions(PythonRDD.scala:65)
	at org.apache.spa

In [22]:
df.write.partitionBy("partition").mode("overwrite").parquet("ingest-data")